In [1]:
import pandas as pd
import os
import pyopencl as pcl
import numpy as np

In [3]:
cl_code_file = os.path.join('..', 'src', 'lr_opencl', 'cl', 'logistic.cl')
os.path.isfile(cl_code_file)

True

In [4]:
# let's try to find available devices
platforms = pcl.get_platforms()
for p in platforms:
    devs = p.get_devices()
    for d in devs:
        print(d.name, pcl.device_type.to_string(d.type), d.global_mem_size / 10**9)

# let's select the AMD radeon card in this case
dev=None
for p in pcl.get_platforms():
    devs = p.get_devices()
    for d in devs:
        if pcl.device_type.to_string(d.type) == 'GPU' and (d.global_mem_size / 10**9) > 2.0:
            dev = d
            
# make the opencl context
# cntx = pcl.create_some_context()
cntx = pcl.Context(devices=[dev])
queue = pcl.CommandQueue(cntx, device=dev)

Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GHz CPU 17.179869184
Iris Pro GPU 1.610612736
AMD Radeon R9 M370X Compute Engine GPU 2.147483648


In [5]:
# build the kernel
with open(cl_code_file, 'r') as f:
    programs = pcl.Program(cntx, f.read()).build()
    f.seek(0)
    print(f.read())

RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - 

Build on <pyopencl.Device 'AMD Radeon R9 M370X Compute Engine' on 'Apple' at 0x1021c00>:

<program source>:3:7: warning: no previous prototype for function 'sigmoid'
float sigmoid(__global float * X, __global float * theta, const int size)
      ^
<program source>:5:22: warning: implicit declaration of function 'dotproduct' is invalid in C99
  float linear_sum = dotproduct(X, theta, size);
                     ^
<program source>:11:49: warning: type specifier missing, defaults to 'int'
float dotproduct(__global float * a, __global * float b, const int size)
                                     ~~~~~~~~   ^
<program source>:11:49: error: expected ')'
<program source>:11:17: note: to match this '('
float dotproduct(__global float * a, __global * float b, const int size)
                ^
<program source>:11:7: error: conflicting types for 'dotproduct'
float dotproduct(__global float * a, __global * float b, const int size)
  